In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the DCCA model
class DCCA(nn.Module):
    def __init__(self, input_size1, input_size2, n_components, layer_sizes1, layer_sizes2):
        super(DCCA, self).__init__()
        self.input_size1 = input_size1
        self.input_size2 = input_size2
        self.n_components = n_components
        self.layer_sizes1 = layer_sizes1
        self.layer_sizes2 = layer_sizes2

        # Define the first view network
        self.net1 = nn.Sequential()
        for i, layer_size in enumerate(self.layer_sizes1):
            if i == 0:
                self.net1.add_module('linear{}'.format(i), nn.Linear(self.input_size1, layer_size))
            else:
                self.net1.add_module('linear{}'.format(i), nn.Linear(layer_sizes1[i-1], layer_size))
            self.net1.add_module('relu{}'.format(i), nn.ReLU())

        # Define the second view network
        self.net2 = nn.Sequential()
        for i, layer_size in enumerate(self.layer_sizes2):
            if i == 0:
                self.net2.add_module('linear{}'.format(i), nn.Linear(self.input_size2, layer_size))
            else:
                self.net2.add_module('linear{}'.format(i), nn.Linear(layer_sizes2[i-1], layer_size))
            self.net2.add_module('relu{}'.format(i), nn.ReLU())

        # Define the correlation layer
        self.correlation = nn.Linear(self.n_components, self.n_components)

    def forward(self, x1, x2):
        # Forward pass through the first view network
        x1 = self.net1(x1)

        # Forward pass through the second view network
        x2 = self.net2(x2)

        # Compute the correlation between the two views
        correlation = torch.matmul(x1, x2.t())
        correlation = self.correlation(correlation)

        return correlation

# Define the training function
def train(model, train_loader, optimizer, criterion):
    model.train()
    for batch_idx, (data1, data2) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data1, data2)
        loss = criterion(output, torch.eye(output.size(0)).cuda())
        loss.backward()
        optimizer.step()

# Define the testing function
def test(model, test_loader):
    model.eval()
    with torch.no_grad():
        for batch_idx, (data1, data2) in enumerate(test_loader):
            output = model(data1, data2)
            print(output)

# Define the hyperparameters
input_size1 = 784
input_size2 = 784
n_components = 10
layer_sizes1 = [256, 256, 10]
layer_sizes2 = [256, 256, 10]
batch_size = 100
num_epochs = 100
learning_rate = 0.001


In [10]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
# Load the digits dataset
digits = load_digits()

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2, random_state=42)


# Define the train dataset for DCCA
train_dataset = (X_train, X_train)
test_dataset = (X_test, y_test)

In [12]:
import torch
# Load the data
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=20, shuffle=False)

# Initialize the model, optimizer, and criterion
model = DCCA(input_size1, input_size2, n_components, layer_sizes1, layer_sizes2)
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

# Train the model
for epoch in range(num_epochs):
    train(model, train_loader, optimizer, criterion)

# Test the model
test(model, test_loader)

RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float